In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
import tensorflow
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')
import sklearn.metrics as metrics
import warnings



from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score
from xgboost import plot_importance
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE



C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_selection.univariate_selection module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
Child = pd.read_csv("./Child_new4_score.csv",encoding = 'CP949')
Child

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4,21,22,25,29,31,44,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,개별사건번호,피해아동대상자,학대행위자대상,아동_성별,아동_연령대,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,...,아조_원가정보호,아조_가정복귀,행조_아동과의분리,행조_만나지못함,행조_모니터링,행조_고소고발사건처,행조_사건처리만나지,New_Abuse_Count,New_재학대혐의여부,New_Call_Count
0,1.020000e+20,CASRCP20121010398,CASRCP20121020314,M,NaN,내국인,NaN,NaN,자택,양부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0
1,8.040000e+20,CASRCP20150110545,CASRCP20150120502,M,NaN,내국인,NaN,NaN,기타,친부모,...,1,0,0,0,1,0,0,0.0,N,0.0
2,4.020000e+20,CASRCP20141211229,CASRCP20141221114,M,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,NaN,NaN,0.0
3,4.020000e+20,CASRCP20141211178,CASRCP20141221114,F,NaN,내국인,NaN,NaN,NaN,친부,...,0,0,0,0,0,0,0,NaN,NaN,0.0
4,1.060000e+20,CASRCP20141211459,CASRCP20141221403,F,NaN,내국인,NaN,NaN,보증금(전세)+월세,친부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122030,5.800000e+21,CASRCP20180100595,CASRCP20180100597,M,1~3세,내국인,NaN,어린이집,NaN,친모,...,1,0,0,0,1,0,0,0.0,N,0.0
122031,5.120000e+21,CASRCP20180100289,CASRCP20180100290,F,13~15세,내국인,재학중,초등학교,NaN,친모,...,1,0,0,0,1,0,0,0.0,N,0.0
122032,6.400000e+21,CASRCP20180102065,CASRCP20180102067,F,NaN,NaN,NaN,NaN,NaN,친부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0
122033,5.720000e+21,CASRCP20180101264,CASRCP20180101266,M,1~3세,내국인,재학중,어린이집,NaN,친부모,...,0,0,0,0,0,0,0,NaN,NaN,0.0


In [3]:
for i in range(len(Child.columns)):
    print(i, Child.columns[i])

0 개별사건번호
1 피해아동대상자
2 학대행위자대상
3 아동_성별
4 아동_연령대
5 아동_내국인여부
6 아동_최종학력
7 아동_직업유형
8 아동_거주상태
9 아동_친권자유형
10 아동_가족유형
11 아동_다문화가족
12 아동_가구소득구분코
13 아동_기초생활수급유
14 아동_주요병력
15 아동_보호조치유형
16 신고_접수경로구분코
17 신고_접수일시
18 신고_신고자유형구분
19 신고_통보경찰소속기
20 신고_집단시설내사건
21 신고_재신고여부_1
22 신고_재신고유형
23 신고_재신고이전접수
24 신고_접수유형
25 신고_관할지구대
26 신고_신고접수구분
27 신고_접수자소속기관
28 신고_관할지역ID
29 신고_관련키워드
30 신고_피해아동상태구
31 신고_재신고센터ID
32 신고_관할지역기관I
33 신고_관리센터ID
34 신대_행위자아동관계
35 신대_아동동거여부
36 신대_접수유형
37 신대_재신고여부
38 신대_재신고유형_1
39 신대_재신고센터ID
40 신대_재신고이전
41 신대_통계거점
42 신대_통계기관
43 신대_통계관할지역
44 조사_집단시설내사건
45 조사_조사완료여부
46 조사_관할지역ID_
47 조사_관할지역기관I
48 조사_위험_점수
49 조사_위험_유형
50 조대_재신고여부_1
51 조대_재신고유형구분
52 조대_재신고관리기관
53 조대_재신고이전사건
54 조대_동일신고여부
55 조대_아동동거여부
56 조대_특별관리유형
57 판단_학대혐의여부
58 판단_유형_신체
59 판단_유형_정서
60 판단_유형_성
61 판단_유형_방임
62 판단_유형_유기
63 판단_학대발생빈도
64 결과_조치결과일자
65 신고_접수연도
66 아특_장애의심
67 아특_신체질환및장애
68 아특_장루요루장애
69 아특_난작문
70 아특_애착문제
71 아특_탐식및결식
72 아특_시각장애
73 아특_신체발달지연
74 아특_부부및가족갈등
75 아특_종교문제
76 아특_어릴적학대경험
77 아특_특성없음
78 아특_공격성
79 아특_성격및기질문제
80 아특_정서문제
81

In [4]:
Child.columns[49]

'조사_위험_유형'

In [5]:
Child_score = Child['조사_위험_유형']
Child_score

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
122030            NaN
122031          01,03
122032    03,09,06,05
122033              7
122034              7
Name: 조사_위험_유형, Length: 122035, dtype: object

In [6]:
Child_score = Child_score.replace('1','01')
Child_score = Child_score.replace('2','02')
Child_score = Child_score.replace('3','03')
Child_score = Child_score.replace('4','04')
Child_score = Child_score.replace('5','05')
Child_score = Child_score.replace('6','06')
Child_score = Child_score.replace('7','07')
Child_score = Child_score.replace('8','08')
Child_score = Child_score.replace('9','09')
Child_score

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
122030            NaN
122031          01,03
122032    03,09,06,05
122033             07
122034             07
Name: 조사_위험_유형, Length: 122035, dtype: object

In [7]:
Child['조사_위험_유형'] = Child_score

In [8]:
Child['조사_위험_유형'].value_counts()

07                   2382
03                   2333
01,03                2273
01,05,03             1786
01,07,03             1382
                     ... 
02,09,08,06,05,03       1
01,08,06,04,02          1
01,09,05,06             1
01,07,06,05,04          1
3.0                     1
Name: 조사_위험_유형, Length: 422, dtype: int64

In [9]:
Child.to_csv('./Child_new5.csv', encoding='cp949', index=False)

In [10]:
sc = []
count =0

for i in range(len(Child_score)):
    try:
        for j in range(len(Child_score[i].split(','))):
            sc.append(Child_score[i].split(',')[j])
    except AttributeError:
        sc.append(np.nan)
    count+=1
    
print(count)
print(len(sc))
sc

122035
206645


[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [11]:
s = pd.Series(sc)    # 시리즈화
s.value_counts()     # 특성별 개수파악

03    30441
01    22461
07    18817
08    15079
05    14369
09    12895
06     6972
02     4148
04     3007
dtype: int64

In [12]:
sc_value = list(set(sc)) # sc의 구성요소가 무엇인지 뽑아내기
sc_value

[nan, '08', '06', '03', '09', '04', '01', '05', '02', '07']